In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
disaster = pd.read_csv('C:/Users/Dikshant Gupta/Desktop/VIT/SEM 6/CSE3020_DV_Prj_NIL/data/tweets.csv')
disaster.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [3]:
# Loading required features
disaster = disaster[['text','target']]
disaster.head()

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0


In [6]:
disaster['text'] = [entry.lower() for entry in disaster['text']]
disaster['text'].head()

0    communal violence in bhainsa, telangana. "ston...
1    telangana: section 144 has been imposed in bha...
2    arsonist sets cars ablaze at dealership https:...
3    arsonist sets cars ablaze at dealership https:...
4    "lord jesus, your love brings freedom and pard...
Name: text, dtype: object

In [7]:
# Loading packages for Tokenzation
import nltk
from nltk.tokenize import word_tokenize

In [9]:
disaster['text'] = [word_tokenize(entry) for entry in disaster['text']]

In [10]:
disaster['text'].head()

0    [communal, violence, in, bhainsa, ,, telangana...
1    [telangana, :, section, 144, has, been, impose...
2    [arsonist, sets, cars, ablaze, at, dealership,...
3    [arsonist, sets, cars, ablaze, at, dealership,...
4    [``, lord, jesus, ,, your, love, brings, freed...
Name: text, dtype: object

In [11]:
# POS & Stemming
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet

In [12]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['j'] = wn.ADJ
tag_map['v'] = wn.VERB
tag_map['v'] = wn.ADV

In [13]:
# Storing all the stopwords into variables
stop_words = set(stopwords.words("english"))
print(stop_words)

{'but', 'whom', "isn't", "shouldn't", "couldn't", 'be', "shan't", 'does', 'wasn', 'after', 'if', 'his', 'd', 'he', 'wouldn', "wasn't", 's', "hasn't", 'while', 'few', 'during', 'own', 'how', 'no', 'themselves', 'weren', 'did', 'myself', "won't", 'she', 'her', 'on', 'off', 'those', 'will', "doesn't", 'himself', 'nor', 'up', 'an', 'about', "aren't", 'out', 'what', 'isn', 'do', 'only', 'him', 'by', 'both', 'which', 'is', 'again', 'as', 'itself', 'yourself', "don't", 'where', "you've", 'now', 'shouldn', 't', 'through', 'were', 'been', 'such', "should've", 'ma', 'some', 'hasn', 'hers', "needn't", "mustn't", 'didn', 'was', 'your', 're', 'into', 'they', 'for', 'than', 'we', 'them', 'between', 'i', 'won', 'theirs', 'it', 'don', 'y', 'why', 'of', 'with', 'having', 'then', 'these', 'had', 'each', "wouldn't", 'not', 'doesn', 'so', 'you', 'my', "you're", 'there', 'can', 'am', 'a', 'needn', 'from', 'being', 'herself', 'more', 'have', 'their', 'at', 'are', 'll', 'until', 'that', 'hadn', 'over', 'just

In [14]:
for index,entry in enumerate(disaster['text']):
    Final_words = []
    word_lemmstized = WordNetLemmatizer()
    for word,tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_lemmstized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_final)
    disaster.loc[index,'text_final'] = str(Final_words)

In [15]:
disaster.head()

,text,target,text_final
0,"[communal, violence, in, bhainsa, ,, telangana...",1,"['communal', 'violence', 'bhainsa', 'telangana..."
1,"[telangana, :, section, 144, has, been, impose...",1,"['telangana', 'section', 'imposed', 'bhainsa',..."
2,"[arsonist, sets, cars, ablaze, at, dealership,...",1,"['arsonist', 'set', 'car', 'ablaze', 'dealersh..."
3,"[arsonist, sets, cars, ablaze, at, dealership,...",1,"['arsonist', 'set', 'car', 'ablaze', 'dealersh..."
4,"[``, lord, jesus, ,, your, love, brings, freed...",0,"['lord', 'jesus', 'love', 'brings', 'freedom',..."


# LSTM Based Model Building & Testing

In [32]:
from keras.models import Model
from keras import preprocessing
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,naive_bayes
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.utils import pad_sequences
import keras
%matplotlib inline

In [33]:
Train_X,Test_X,Train_Y,Test_Y = sklearn.model_selection.train_test_split(disaster['text_final'],disaster['target'],test_size = 0.3)
encoder = LabelEncoder()
Train_Y = encoder.fit_transform(Train_Y)
Test_Y = encoder.fit_transform(Test_Y)

Tfidf_vect = TfidfVectorizer(max_features = 5000)
Tfidf_vect.fit(disaster['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Train_X)
sequences = tok.texts_to_sequences(Train_X)
sequences_matrix = keras.utils.pad_sequences(sequences,maxlen=max_len)

In [34]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [36]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [61]:
history = model.fit(sequences_matrix,Train_Y,batch_size=128,epochs=20,validation_split=0.2)

Epoch 1/20
50/50 [==============================] - 5s 94ms/step - loss: 0.0073 - accuracy: 0.9978 - val_loss: 3.4840 - val_accuracy: 0.7927
Epoch 2/20
50/50 [==============================] - 5s 102ms/step - loss: 0.0079 - accuracy: 0.9973 - val_loss: 3.6460 - val_accuracy: 0.7877
Epoch 3/20
50/50 [==============================] - 5s 108ms/step - loss: 0.0076 - accuracy: 0.9970 - val_loss: 3.6216 - val_accuracy: 0.8028
Epoch 4/20
50/50 [==============================] - 5s 93ms/step - loss: 0.0075 - accuracy: 0.9970 - val_loss: 3.9567 - val_accuracy: 0.8053
Epoch 5/20
50/50 [==============================] - 5s 100ms/step - loss: 0.0076 - accuracy: 0.9969 - val_loss: 3.7718 - val_accuracy: 0.8040
Epoch 6/20
50/50 [==============================] - 6s 111ms/step - loss: 0.0067 - accuracy: 0.9975 - val_loss: 3.7531 - val_accuracy: 0.8059
Epoch 7/20
50/50 [==============================] - 5s 108ms/step - loss: 0.0077 - accuracy: 0.9972 - val_loss: 3.8260 - val_accuracy: 0.8028
Epoch 8/

In [62]:
test_sequences = tok.texts_to_sequences(Test_X)
test_sequences_matrix = keras.utils.pad_sequences(test_sequences,maxlen=max_len)

In [63]:
accr = model.evaluate(test_sequences_matrix,Test_Y)

107/107 [==============================] - 2s 19ms/step - loss: 2.9150 - accuracy: 0.8030


In [64]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 2.915
  Accuracy: 0.803


In [65]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history_LSTM_20epoch.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [66]:
import pickle
# save the model to disk
filename = 'LSTM_80.30acc.sav'
pickle.dump(model, open(filename, 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\activation
......vars
...layers\activation_1
......vars
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\embedding
......vars
.........0
...layers\input_layer
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........2
.........3
.........4
.........5
.........6
.........7
.........8
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-29 15:48:10         4104
metadata.json                                  2023-03-29 15:48:10           64
variables.h5                                   2023-03-29 15:4

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)